## Longest Transcript Only (for orthofinder)

For orthofinder we are already putting in only the primary assembly, so it would be good to get the longest one from that. 

In [22]:
%matplotlib inline

In [23]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import Bio
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqUtils import IsoelectricPoint as IP

In [24]:
outpath = "/home/jemimah/databases/orthofinder_input/"
infn_fasta = outpath + "W_ceracea.fa"
brakerinfn = "/home/jemimah/analysis/v3/braker2/20200709_real/braker.gtf.reformatted"

First **identify if all transcripts have a .tX format**

In [25]:
braker_df = pd.read_csv(brakerinfn, sep = '\t')
geneless = braker_df[(braker_df["feature"] != "gene") & (braker_df["feature"] != "transcript")]
geneless.drop_duplicates(subset ="transcript_id", keep = "first", inplace = True)
geneless.sort_values("transcript_id", inplace = True)

pairs = geneless.copy()
del pairs["source"]
del pairs["start"]
del pairs["feature"]
del pairs["end"]
del pairs["score"]
del pairs["strand"]
del pairs["frame"]
del pairs["gene_id"]
pri_pairs = pairs[~pairs["seqname"].str.contains("hap_")]
pri_pairs.reset_index(level=0, inplace=True)
pri_pairs

<ipython-input-25-78dfe0a1c7db>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geneless.drop_duplicates(subset ="transcript_id", keep = "first", inplace = True)
<ipython-input-25-78dfe0a1c7db>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geneless.sort_values("transcript_id", inplace = True)


,index,seqname,transcript_id
0,709884,tig00007534,file_1_file_1_g1000.t1
1,199299,tig00007534,file_1_file_1_g1001.t1
2,788865,tig00007534,file_1_file_1_g1002.t1
3,1005830,tig00007534,file_1_file_1_g1003.t1
4,496937,tig00007534,file_1_file_1_g1004.t1
...,...,...,...
51937,723805,tig00013813,file_2_75741_t
51938,64026,tig00013813,file_2_7695_t
51939,1065120,tig00005423,file_2_77904_t
51940,985933,tig00001276,file_2_8500_t


We cannot use the start/end lengths in the braker table to select longest cds, as coding sequences have been combined in the braker2.aa result. So the first step is to figure out how many transcripts appear more than once in the primary assembly.

In [26]:
len(pri_pairs.transcript_id)

51942

In [27]:
len(pri_pairs.transcript_id.str.rstrip('t1234567890').unique())

50225

In [28]:
51942 - 50225

1717

So there are 1717 things from the primary assembly that can be removed. The below list of transcript ids is what we want to check.

In [29]:
list(pri_pairs.transcript_id.str.rstrip('t1234567890').unique())

['file_1_file_1_g1000.',
 'file_1_file_1_g1001.',
 'file_1_file_1_g1002.',
 'file_1_file_1_g1003.',
 'file_1_file_1_g1004.',
 'file_1_file_1_g10044.',
 'file_1_file_1_g10045.',
 'file_1_file_1_g10046.',
 'file_1_file_1_g10047.',
 'file_1_file_1_g10048.',
 'file_1_file_1_g10049.',
 'file_1_file_1_g1005.',
 'file_1_file_1_g10050.',
 'file_1_file_1_g10051.',
 'file_1_file_1_g10052.',
 'file_1_file_1_g10053.',
 'file_1_file_1_g10054.',
 'file_1_file_1_g10055.',
 'file_1_file_1_g10056.',
 'file_1_file_1_g10057.',
 'file_1_file_1_g10058.',
 'file_1_file_1_g10059.',
 'file_1_file_1_g1006.',
 'file_1_file_1_g10060.',
 'file_1_file_1_g10061.',
 'file_1_file_1_g10062.',
 'file_1_file_1_g10067.',
 'file_1_file_1_g10068.',
 'file_1_file_1_g10069.',
 'file_1_file_1_g1007.',
 'file_1_file_1_g1008.',
 'file_1_file_1_g1009.',
 'file_1_file_1_g1010.',
 'file_1_file_1_g1011.',
 'file_1_file_1_g1012.',
 'file_1_file_1_g10126.',
 'file_1_file_1_g10127.',
 'file_1_file_1_g10128.',
 'file_1_file_1_g10129.',

In [30]:
pri_stripped = dict(pri_pairs.transcript_id.str.rstrip('t1234567890').value_counts())

In [31]:
#pri_stripped

So now pri_stripped is a dictionary of the stripped values and how often they occour.

Probably best idea is to import the pri walli fasta file through biopython, then run it through a for loop with this list.

In [32]:
records = list(SeqIO.parse(infn_fasta, "fasta"))

So we can check everything in the record against pri_stripped. If its there once: keep the record as is, add to a "keep" list.  
The issue if its there >once, is that there isn't a way to search the list I dont think?

In [33]:
ones, twos, threes, fours, fives, sixes = 0, 0, 0, 0, 0, 0

for x in range(1,len(records)):
    if (pri_stripped[records[x].id.rstrip('t1234567890')]) < 1:
        print("got a 0???")
    elif (pri_stripped[records[x].id.rstrip('t1234567890')]) == 1:
        ones += 1
    elif (pri_stripped[records[x].id.rstrip('t1234567890')]) == 2:
        twos += 1
    elif (pri_stripped[records[x].id.rstrip('t1234567890')]) == 3:
        threes += 1
    elif (pri_stripped[records[x].id.rstrip('t1234567890')]) == 4:
        fours += 1
    elif (pri_stripped[records[x].id.rstrip('t1234567890')]) == 5:
        fives += 1
    elif (pri_stripped[records[x].id.rstrip('t1234567890')]) == 6:
        sixes += 1
    else:
        print(pri_stripped[records[x].id.rstrip('t1234567890')])
        
print("ones =", ones, "\ntwos =", twos, "\nthrees =", threes, "\nfours =", fours, "\nfives =", fives, "\nsixes =", sixes)

ones = 48670 
twos = 2814 
threes = 402 
fours = 44 
fives = 5 
sixes = 6


In [34]:
keep_records_2 = []
same_lengths = []

for key in pri_stripped:
    match_records = []
    key_lengths = []
    for x in range(0,(len(records))):
        if records[x].id.rstrip('t1234567890') == key:
            match_records += [records[x]]
            key_lengths += [len(records[x].seq)]
    if len(match_records) == 1:
        keep_records_2 += [match_records]
    elif len(match_records) < 1:
        print("oh no < 1", key)
    else:
        key_matches = zip(match_records, key_lengths)
        max_length = max(key_lengths)
        keep_records_temp = []
        for match, length in key_matches:
            if length == max_length:
                keep_records_temp += [match]
        
        if len(keep_records_temp) == 1:
            keep_records_2 += [keep_records_temp]
        else:
            same_lengths += [keep_records_temp]
print(len(keep_records_2), len(same_lengths))

50201 24


In [35]:
len(pri_stripped)

50225

So there is an output for each memeber of stripped - as there should be. We did not keep track of rejected records, so I can't check that all the numbers add up to len(records).  

Everything in same_length was 2 items long:

In [36]:
for x in same_lengths:
    if len(x) != 2:
        print(">2")

And as there were only 24, I checked them all and they were all t1 and t2 (I guess that the shorter ones get later numbers?), and decided we could just go with .t1

In [37]:
for x in same_lengths:
    print(len(x))
    for y in x:
        print(y.id)

2
file_1_file_1_g59336.t2
file_1_file_1_g59336.t1
2
file_1_file_1_g58502.t2
file_1_file_1_g58502.t1
2
file_1_file_1_g1788.t1
file_1_file_1_g1788.t2
2
file_1_file_1_g85029.t1
file_1_file_1_g85029.t2
2
file_1_file_1_g28230.t2
file_1_file_1_g28230.t1
2
file_1_file_1_g80471.t1
file_1_file_1_g80471.t2
2
file_1_file_1_g21480.t2
file_1_file_1_g21480.t1
2
file_1_file_1_g47304.t2
file_1_file_1_g47304.t1
2
file_1_file_1_g66009.t2
file_1_file_1_g66009.t1
2
file_1_file_1_g40930.t2
file_1_file_1_g40930.t1
2
file_1_file_1_g81899.t1
file_1_file_1_g81899.t2
2
file_1_file_1_g80086.t2
file_1_file_1_g80086.t1
2
file_1_file_1_g77976.t2
file_1_file_1_g77976.t1
2
file_1_file_1_g25233.t1
file_1_file_1_g25233.t2
2
file_1_file_1_g43213.t2
file_1_file_1_g43213.t1
2
file_1_file_1_g64815.t1
file_1_file_1_g64815.t2
2
file_1_file_1_g49201.t1
file_1_file_1_g49201.t2
2
file_1_file_1_g25956.t1
file_1_file_1_g25956.t2
2
file_1_file_1_g21353.t2
file_1_file_1_g21353.t1
2
file_1_file_1_g84309.t2
file_1_file_1_g84309.t1
2


This one takes FOREVER so is hashed out

In [38]:
keep_records_2 = []

#for each key
for key in pri_stripped:
    match_records = []
    key_lengths = []
    
#find all the records which match the given key
    for x in range(0,(len(records))):
        if records[x].id.rstrip('t1234567890') == key:
            match_records += [records[x]]
            key_lengths += [len(records[x].seq)]
            
#move on if there is only one match (and error if <1)
    if len(match_records) == 1:
        keep_records_2 += match_records
    elif len(match_records) < 1:
        print("oh no < 1", key)

#if there are multiple maches, find all which are the longest length
    else:
        key_matches = zip(match_records, key_lengths)
        max_length = max(key_lengths)
        keep_records_temp = []
        for match, length in key_matches:
            if length == max_length:
                keep_records_temp += [match]
                
#move on if thats only one thing
        if len(keep_records_temp) == 1:
            keep_records_2 += keep_records_temp

#or take the .t1 if its more than one.
        else:
            t1 = keep_records_temp[0].id.rstrip('t1234567890') + "t1"
            for y in keep_records_temp:
                if y.id == t1:
                    keep_records_2 += [y]
print(len(keep_records_2))

50225


In [39]:
keep_records_ids = []
for record in keep_records_2:
    keep_records_ids += [record.id]
print(keep_records_ids[:10])

['file_1_file_1_g5851.t6', 'file_1_file_1_g83002.t4', 'file_1_file_1_g7707.t3', 'file_1_file_1_g20541.t3', 'file_1_file_1_g19219.t4', 'file_1_file_1_g33615.t3', 'file_1_file_1_g26460.t2', 'file_1_file_1_g70243.t2', 'file_1_file_1_g69049.t3', 'file_1_file_1_g26794.t1']


so then I think keep_records_2 is the records. And keep_records_ids is just the ids. I dont really like using biopython to export a fasta file, so I'm gonna use bash again.

In [40]:
joined_ids = "\n".join(keep_records_ids) + "\n"

In [41]:
with open((outpath + "longest_cds_pri.txt"), "w") as f:
    f.write(joined_ids)

In [42]:
%%bash -s "$outpath"
cd $1
seqtk subseq W_ceracea_both.fa longest_cds_pri.txt > W_ceracea_longest.fa
grep -c ">" W_ceracea_longest.fa

50225


In [43]:
len(pri_stripped)

50225

So assuming pri_stripped is right, W_ceracea_longest.fa is right.